# Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Teste planejados:
  1. Teste com diferentes dataset (Somente normalizado, Nomalizado e oversampling e normalizado e under sampling)
  2. Teste de hiperparametros

Qual vai ser a metodologia? Primeiro vou testar os 3 datasets com a condições default para ter a baseline e depois vou testar os 3 com os melhores parametros achados pelo RandomizedSearch 2 vezes (2 vezes buscando os melhores resultados)

In [ ]:
import pandas as pd

In [ ]:
df_norm = pd.read_csv('drive/MyDrive/Aprendizagem de máquina/Segundo Projeto/datasets/data_normalizada.csv')
df_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 94 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Bankrupt?                                                6819 non-null   int64  
 1   ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2   ROA(A) before interest and % after tax                   6819 non-null   float64
 3   ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4   Operating Gross Margin                                   6819 non-null   float64
 5   Realized Sales Gross Margin                              6819 non-null   float64
 6   Operating Profit Rate                                    6819 non-null   float64
 7   Pre-tax net Interest Rate                                6819 non-null   float64
 8   After-tax net Interest Rate 

In [ ]:
df_over = pd.read_csv('drive/MyDrive/Aprendizagem de máquina/Segundo Projeto/datasets/data_smote.csv')
df_over.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13198 entries, 0 to 13197
Data columns (total 94 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Bankrupt?                                                13198 non-null  int64  
 1   ROA(C) before interest and depreciation before interest  13198 non-null  float64
 2   ROA(A) before interest and % after tax                   13198 non-null  float64
 3   ROA(B) before interest and depreciation after tax        13198 non-null  float64
 4   Operating Gross Margin                                   13198 non-null  float64
 5   Realized Sales Gross Margin                              13198 non-null  float64
 6   Operating Profit Rate                                    13198 non-null  float64
 7   Pre-tax net Interest Rate                                13198 non-null  float64
 8   After-tax net Interest Rat

In [ ]:
df_under = pd.read_csv('drive/MyDrive/Aprendizagem de máquina/Segundo Projeto/datasets/data_under.csv')
df_under.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 94 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Bankrupt?                                                440 non-null    int64  
 1   ROA(C) before interest and depreciation before interest  440 non-null    float64
 2   ROA(A) before interest and % after tax                   440 non-null    float64
 3   ROA(B) before interest and depreciation after tax        440 non-null    float64
 4   Operating Gross Margin                                   440 non-null    float64
 5   Realized Sales Gross Margin                              440 non-null    float64
 6   Operating Profit Rate                                    440 non-null    float64
 7   Pre-tax net Interest Rate                                440 non-null    float64
 8   After-tax net Interest Rate   

Dividindo entre treino e teste

# Baselines

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split

def resultados(nome = ''):
  dataset_atual = []
  if nome == '':
    dataset_atual = df_norm
  elif nome == 'over':
    dataset_atual = df_over
  elif nome == 'under':
    dataset_atual = df_under

  y = dataset_atual['Bankrupt?']

  # Removendo a coluna alvo do DataFrame original para obter apenas as features
  X = dataset_atual.drop(columns=['Bankrupt?'])

  # Dividindo os dados em conjuntos de treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  #KNN
  knn_classifier = KNeighborsClassifier()
  knn_classifier.fit(X_train, y_train)

  # MLP
  mlp_classifier = MLPClassifier()
  mlp_classifier.fit(X_train, y_train)

  # KNN
  y_pred_knn = knn_classifier.predict(X_test)

  # MLP
  y_pred_mlp = mlp_classifier.predict(X_test)

  # Comitê de Redes Neurais
  model1 = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000)
  model2 = MLPClassifier(hidden_layer_sizes=(50, 20), max_iter=1000)
  model3 = MLPClassifier(hidden_layer_sizes=(80, 30), max_iter=1000)

  # O Voting Classifier funciona com um simples voto de maioria
  committee = VotingClassifier(estimators=[('mlp1', model1), ('mlp2', model2), ('mlp3', model3)], voting='hard')
  committee.fit(X_train, y_train)
  y_pred_com = committee.predict(X_test)

  # Avaliação do KNN
  print(f"Relatório de Classificação para KNN {nome}:")
  print(classification_report(y_test, y_pred_knn))

  print('')
  # Avaliação do MLP
  print(f"Relatório de Classificação para MLP {nome}:")
  print(classification_report(y_test, y_pred_mlp))

  print('')
  # Avaliação do comitê de Redes Neurais
  print(f"Relatório de Classificação para Comitê de NN {nome}:")
  print(classification_report(y_test, y_pred_com))

## Dataset normalizado

Usando os modelos nas condições default

In [ ]:
resultados()

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Relatório de Classificação para KNN :
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1313
           1       0.55      0.12      0.19        51

    accuracy                           0.96      1364
   macro avg       0.76      0.56      0.59      1364
weighted avg       0.95      0.96      0.95      1364


Relatório de Classificação para MLP :
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1313
           1       0.36      0.27      0.31        51

    accuracy                           0.95      1364
   macro avg       0.67      0.63      0.64      1364
weighted avg       0.95      0.95      0.95      1364


Relatório de Classificação para Comitê de NN :
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1313
           1       0.38      0.22      0.28        51

    accuracy                           0.96      1364
   

Dá pra notar que a classe 1 não está sendo muito bem reconhecida!

## Dataset Over Sampling

In [ ]:
resultados(nome="over")

Relatório de Classificação para KNN over:
              precision    recall  f1-score   support

           0       1.00      0.87      0.93      1334
           1       0.88      1.00      0.94      1306

    accuracy                           0.94      2640
   macro avg       0.94      0.94      0.94      2640
weighted avg       0.94      0.94      0.94      2640


Relatório de Classificação para MLP over:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99      1334
           1       0.97      1.00      0.99      1306

    accuracy                           0.99      2640
   macro avg       0.99      0.99      0.99      2640
weighted avg       0.99      0.99      0.99      2640


Relatório de Classificação para Comitê de NN over:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1334
           1       0.98      1.00      0.99      1306

    accuracy                           0.99  

Com oversampling os resultados ficaram muito mais positivos! Com o comitê sendo o melhor e o KNN sendo o pior

## Dataset Under Sampling

In [ ]:
resultados(nome="under")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Relatório de Classificação para KNN under:
              precision    recall  f1-score   support

           0       0.89      0.76      0.82        51
           1       0.73      0.86      0.79        37

    accuracy                           0.81        88
   macro avg       0.81      0.81      0.81        88
weighted avg       0.82      0.81      0.81        88


Relatório de Classificação para MLP under:
              precision    recall  f1-score   support

           0       0.85      0.76      0.80        51
           1       0.71      0.81      0.76        37

    accuracy                           0.78        88
   macro avg       0.78      0.79      0.78        88
weighted avg       0.79      0.78      0.79        88


Relatório de Classificação para Comitê de NN under:
              precision    recall  f1-score   support

           0       0.83      0.75      0.78        51
           1       0.69      0.78      0.73        37

    accuracy                           0.7

Os resultados ficaram mais positivos do que o dataset somente normalizado. Apesar disso, ficou um pouco pior do que o oversampling.

# Testes de parametros

## Definindo a 1a grade de procura para o GridSearch

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
parameters_com = {
    'mlp1__hidden_layer_sizes': [(50, 20), (80, 30)],
    'mlp2__hidden_layer_sizes': [(100, 50), (80, 30)],
    'mlp3__hidden_layer_sizes': [(50, 20), (80, 30)]
}

parameters_mlp = {
    'hidden_layer_sizes': [(100, 50), (50, 20), (80, 30)]
}

parameters_knn = {
    'n_neighbors': [2, 3, 5, 7, 10],  # Número de vizinhos
}

## Comitê de NN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
def busca_resultados_com(nome = '', parameters = 0):
  dataset_atual = ''
  hidden_layers_passada_1 = ""
  hidden_layers_passada_2 = ""
  hidden_layers_passada_3 = ""
  if nome == '':
    dataset_atual = df_norm
    if parameters:
      hidden_layers_passada_1 = (80, 30)
      hidden_layers_passada_2 = (80, 30)
      hidden_layers_passada_3 = (80, 30)
  elif nome == 'over':
    dataset_atual = df_over
    if parameters:
      hidden_layers_passada_1 = (80, 30)
      hidden_layers_passada_2 = (100, 50)
      hidden_layers_passada_3 = (80, 30)

  elif nome == 'under':
    dataset_atual = df_under
    if parameters:
      hidden_layers_passada_1 = (50, 20)
      hidden_layers_passada_2 = (100, 50)
      hidden_layers_passada_3 = (80, 30)


  y = dataset_atual['Bankrupt?']

  # Removendo a coluna alvo do DataFrame original para obter apenas as features
  X = dataset_atual.drop(columns=['Bankrupt?'])

  # Dividindo os dados em conjuntos de treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Comitê de Redes Neurais
  model1 = MLPClassifier()
  model2 = MLPClassifier()
  model3 = MLPClassifier()
  # Validação cruzada feita aqui também
  parametersAtual = parameters_com
  if parameters == 1:
    parametersAtual = parameters_com_2
    model1 = MLPClassifier(hidden_layer_sizes= hidden_layers_passada_1)
    model2 = MLPClassifier(hidden_layer_sizes= hidden_layers_passada_2)
    model3 = MLPClassifier(hidden_layer_sizes= hidden_layers_passada_3)

  # O Voting Classifier funciona com um simples voto de maioria
  committee = VotingClassifier(estimators=[('mlp1', model1), ('mlp2', model2), ('mlp3', model3)], voting='hard')
  grid_search_com = GridSearchCV(estimator=committee, param_grid=parametersAtual, cv=3)

  grid_search_com.fit(X_train, y_train)
  # Obter os melhores parâmetros encontrados
  best_params = grid_search_com.best_params_
  print("Melhores parâmetros para o comitê de redes neurais:", best_params)
  print('')

  best_model = grid_search_com.best_estimator_
  y_pred = best_model.predict(X_test)
  report = classification_report(y_test, y_pred)
  print("Relatório de Classificação do melhor modelo do comitê de redes neurais:")
  print(report)

## MLP

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
def busca_resultados_mlp(nome = '', parameters = 0):
  dataset_atual = ''
  hidden_layers_passada_1 = ""
  if nome == '':
    dataset_atual = df_norm
    if parameters:
      hidden_layers_passada_1 = (50, 20)
  elif nome == 'over':
    dataset_atual = df_over
    if parameters:
      hidden_layers_passada_1 = (80, 30)

  elif nome == 'under':
    dataset_atual = df_under
    if parameters:
      hidden_layers_passada_1 = (50, 20)

  y = dataset_atual['Bankrupt?']

  # Removendo a coluna alvo do DataFrame original para obter apenas as features
  X = dataset_atual.drop(columns=['Bankrupt?'])

  # Dividindo os dados em conjuntos de treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  mlp = MLPClassifier()
  parametersAtual = parameters_mlp

  if(parameters):
    parametersAtual = parameters_mlp_2
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layers_passada_1)
  # Criar o objeto GridSearchCV
  grid_search = GridSearchCV(estimator=mlp, param_grid=parametersAtual, cv=3)

  # Executar a busca em grade nos dados de treinamento
  grid_search.fit(X_train, y_train)

  # Obter os melhores parâmetros encontrados
  best_params = grid_search.best_params_
  print("Melhores parâmetros encontrados:", best_params)

  best_model = grid_search.best_estimator_
  y_pred = best_model.predict(X_test)
  report = classification_report(y_test, y_pred)
  print("Relatório de Classificação do melhor modelo:")
  print(report)

## KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
def busca_resultados_knn(nome = '', parameters = 0):
  dataset_atual = ''
  n_neighbors_passada = ""
  if nome == '':
    dataset_atual = df_norm
    if parameters:
      n_neighbors_passada = 7
  elif nome == 'over':
    dataset_atual = df_over
    if parameters:
      n_neighbors_passada = 2

  elif nome == 'under':
    dataset_atual = df_under
    if parameters:
      n_neighbors_passada = 10

  y = dataset_atual['Bankrupt?']

  # Removendo a coluna alvo do DataFrame original para obter apenas as features
  X = dataset_atual.drop(columns=['Bankrupt?'])

  # Dividindo os dados em conjuntos de treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  knn = KNeighborsClassifier()
  parametersAtual = parameters_knn

  if(parameters):
    parametersAtual = parameters_knn_2
    knn = KNeighborsClassifier(n_neighbors=n_neighbors_passada)

  # Criar o objeto GridSearchCV
  grid_search = GridSearchCV(estimator=knn, param_grid=parametersAtual, cv=3)

  # Executar a busca em grade nos dados de treinamento
  grid_search.fit(X_train, y_train)

  # Obter os melhores parâmetros encontrados
  best_params = grid_search.best_params_
  print("Melhores parâmetros encontrados:", best_params)

  best_model = grid_search.best_estimator_
  y_pred = best_model.predict(X_test)
  report = classification_report(y_test, y_pred)
  print("Relatório de Classificação do melhor modelo:")
  print(report)

## Resultados para as primeiras grades

### Comitê de NN

In [ ]:
print("Base somente normalizada")
busca_resultados_com()
print("")
print("Base com oversampling")
busca_resultados_com(nome="over")
print("")
print("Base com undersampling")
busca_resultados_com(nome="under")

Base somente normalizada
Melhores parâmetros para o comitê de redes neurais: {'mlp1__hidden_layer_sizes': (80, 30), 'mlp2__hidden_layer_sizes': (80, 30), 'mlp3__hidden_layer_sizes': (80, 30)}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1313
           1       0.45      0.25      0.33        51

    accuracy                           0.96      1364
   macro avg       0.71      0.62      0.65      1364
weighted avg       0.95      0.96      0.96      1364


Base com oversampling
Melhores parâmetros para o comitê de redes neurais: {'mlp1__hidden_layer_sizes': (80, 30), 'mlp2__hidden_layer_sizes': (100, 50), 'mlp3__hidden_layer_sizes': (80, 30)}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1334
           1       0.98      1.00  

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Melhores parâmetros para o comitê de redes neurais: {'mlp1__hidden_layer_sizes': (50, 20), 'mlp2__hidden_layer_sizes': (100, 50), 'mlp3__hidden_layer_sizes': (80, 30)}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
              precision    recall  f1-score   support

           0       0.84      0.80      0.82        51
           1       0.74      0.78      0.76        37

    accuracy                           0.80        88
   macro avg       0.79      0.79      0.79        88
weighted avg       0.80      0.80      0.80        88



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### MLP

In [ ]:
print("Base somente normalizada")
busca_resultados_mlp()
print("")
print("Base com oversampling")
busca_resultados_mlp(nome="over")
print("")
print("Base com undersampling")
busca_resultados_mlp(nome="under")

Base somente normalizada
Melhores parâmetros encontrados: {'hidden_layer_sizes': (50, 20)}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1313
           1       0.45      0.35      0.40        51

    accuracy                           0.96      1364
   macro avg       0.71      0.67      0.69      1364
weighted avg       0.96      0.96      0.96      1364


Base com oversampling
Melhores parâmetros encontrados: {'hidden_layer_sizes': (80, 30)}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1334
           1       0.97      1.00      0.98      1306

    accuracy                           0.98      2640
   macro avg       0.99      0.99      0.98      2640
weighted avg       0.99      0.98      0.98      2640


Base com undersampling


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Melhores parâmetros encontrados: {'hidden_layer_sizes': (50, 20)}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       0.87      0.78      0.82        51
           1       0.74      0.84      0.78        37

    accuracy                           0.81        88
   macro avg       0.80      0.81      0.80        88
weighted avg       0.81      0.81      0.81        88



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### KNN

In [ ]:
print("Base somente normalizada")
busca_resultados_knn()
print("")
print("Base com oversampling")
busca_resultados_knn(nome="over")
print("")
print("Base com undersampling")
busca_resultados_knn(nome="under")

Base somente normalizada
Melhores parâmetros encontrados: {'n_neighbors': 7}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1313
           1       0.67      0.12      0.20        51

    accuracy                           0.96      1364
   macro avg       0.82      0.56      0.59      1364
weighted avg       0.96      0.96      0.95      1364


Base com oversampling
Melhores parâmetros encontrados: {'n_neighbors': 2}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      1334
           1       0.95      1.00      0.97      1306

    accuracy                           0.97      2640
   macro avg       0.97      0.97      0.97      2640
weighted avg       0.97      0.97      0.97      2640


Base com undersampling
Melhores parâmetros encontrados: {'n_neighbors': 10}
Relatório de Classificação 

## Definindo a 2a grade para procurar

In [ ]:
parameters_com_2 = {
    'mlp1__alpha': [0.001],  # Parâmetro de regularização L2, contando que o 0.0001 ja foi testado na condição default
    'mlp2__alpha': [0.001],
    'mlp3__alpha': [0.001],
    'mlp1__learning_rate_init': [0.001, 0.01],  # Taxa de aprendizado inicial
    'mlp2__learning_rate_init': [0.001, 0.01],
    'mlp3__learning_rate_init': [0.001, 0.01],
}

parameters_mlp_2 = {
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.1]
}

parameters_knn_2 = {
    'weights': ['uniform', 'distance'],  # Peso dado aos vizinhos
    'p': [1, 2]  # Parâmetro de distância (1 para distância de Manhattan, 2 para distância euclidiana)
}

## Resultados para as segundas grades

### Comitê de NN

In [ ]:
print("Base somente normalizada")
busca_resultados_com(parameters=1)
print("")
print("Base com oversampling")
busca_resultados_com(nome="over", parameters=1)
print("")
print("Base com undersampling")
busca_resultados_com(nome="under", parameters=1)

Base somente normalizada
Melhores parâmetros para o comitê de redes neurais: {'mlp1__alpha': 0.001, 'mlp1__learning_rate_init': 0.001, 'mlp2__alpha': 0.001, 'mlp2__learning_rate_init': 0.01, 'mlp3__alpha': 0.001, 'mlp3__learning_rate_init': 0.01}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1313
           1       0.36      0.24      0.29        51

    accuracy                           0.96      1364
   macro avg       0.67      0.61      0.63      1364
weighted avg       0.95      0.96      0.95      1364


Base com oversampling
Melhores parâmetros para o comitê de redes neurais: {'mlp1__alpha': 0.001, 'mlp1__learning_rate_init': 0.001, 'mlp2__alpha': 0.001, 'mlp2__learning_rate_init': 0.01, 'mlp3__alpha': 0.001, 'mlp3__learning_rate_init': 0.001}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
              precision    recall 

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Melhores parâmetros para o comitê de redes neurais: {'mlp1__alpha': 0.001, 'mlp1__learning_rate_init': 0.001, 'mlp2__alpha': 0.001, 'mlp2__learning_rate_init': 0.001, 'mlp3__alpha': 0.001, 'mlp3__learning_rate_init': 0.001}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
              precision    recall  f1-score   support

           0       0.84      0.75      0.79        51
           1       0.70      0.81      0.75        37

    accuracy                           0.77        88
   macro avg       0.77      0.78      0.77        88
weighted avg       0.78      0.77      0.77        88



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### MLP

In [ ]:
print("Base somente normalizada")
busca_resultados_mlp(parameters=1)
print("")
print("Base com oversampling")
busca_resultados_mlp(nome="over", parameters=1)
print("")
print("Base com undersampling")
busca_resultados_mlp(nome="under", parameters=1)

Base somente normalizada
Melhores parâmetros encontrados: {'alpha': 0.0001, 'learning_rate_init': 0.1}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1313
           1       0.00      0.00      0.00        51

    accuracy                           0.96      1364
   macro avg       0.48      0.50      0.49      1364
weighted avg       0.93      0.96      0.94      1364


Base com oversampling


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Melhores parâmetros encontrados: {'alpha': 0.0001, 'learning_rate_init': 0.001}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1334
           1       0.98      1.00      0.99      1306

    accuracy                           0.99      2640
   macro avg       0.99      0.99      0.99      2640
weighted avg       0.99      0.99      0.99      2640


Base com undersampling


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Melhores parâmetros encontrados: {'alpha': 0.0001, 'learning_rate_init': 0.1}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       0.79      0.80      0.80        51
           1       0.72      0.70      0.71        37

    accuracy                           0.76        88
   macro avg       0.76      0.75      0.75        88
weighted avg       0.76      0.76      0.76        88



### KNN

In [ ]:
print("Base somente normalizada")
busca_resultados_knn(parameters=1)
print("")
print("Base com oversampling")
busca_resultados_knn(nome="over", parameters=1)
print("")
print("Base com undersampling")
busca_resultados_knn(nome="under", parameters=1)

Base somente normalizada
Melhores parâmetros encontrados: {'p': 2, 'weights': 'uniform'}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1313
           1       0.67      0.12      0.20        51

    accuracy                           0.96      1364
   macro avg       0.82      0.56      0.59      1364
weighted avg       0.96      0.96      0.95      1364


Base com oversampling
Melhores parâmetros encontrados: {'p': 1, 'weights': 'uniform'}
Relatório de Classificação do melhor modelo:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      1334
           1       0.94      1.00      0.97      1306

    accuracy                           0.97      2640
   macro avg       0.97      0.97      0.97      2640
weighted avg       0.97      0.97      0.97      2640


Base com undersampling
Melhores parâmetros encontrados: {'p': 1, 'weights': 'un

# Resultados

Vale notar que todos os resultados obtidos após a busca de parâmetros de cada modelo, em todos os datasets, foram melhores do que a baseline

## Comitê NN

### Base de dados normalizada

Melhor Resultado foi realmente os parâmetros utilizados no 1o GridSearch e com os outros parâmetros em padrão. O Resultado foi o seguinte:



Melhores parâmetros para o comitê de redes neurais:
- {'mlp1__hidden_layer_sizes': (80, 30), 'mlp2__hidden_layer_sizes': (80, 30), 'mlp3__hidden_layer_sizes': (80, 30)}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
              
              precision    recall  f1-score   support

               0       0.97      0.99      0.98      1313
               1       0.45      0.25      0.33        51

        accuracy                           0.96      1364
       macro avg       0.71      0.62      0.65      1364
    weighted avg       0.95      0.96      0.96      1364

### Base de dados com oversampling

Já na base com oversampling, todos os resultados foram iguais, uma vez que o modelo já estava desempenhando muito bem em todos os casos nessa base de dados.

Os resultados foram os seguintes:



Melhores parâmetros para o comitê de redes neurais:
- {'mlp1__alpha': 0.001, 'mlp1__learning_rate_init': 0.001, 'mlp2__alpha': 0.001, 'mlp2__learning_rate_init': 0.01, 'mlp3__alpha': 0.001, 'mlp3__learning_rate_init': 0.001}

Relatório de Classificação do melhor modelo do comitê de redes neurais:

              precision    recall  f1-score   support

               0       1.00      0.98      0.99      1334
               1       0.98      1.00      0.99      1306

        accuracy                           0.99      2640
       macro avg       0.99      0.99      0.99      2640
    weighted avg       0.99      0.99      0.99      2640

### Base de dados com undersampling

Nessa database o melhor resultado foi obtido com os parâmetros usados pelo primeiro GridSearch.

Melhores parâmetros para o comitê de redes neurais:
- {'mlp1__hidden_layer_sizes': (50, 20), 'mlp2__hidden_layer_sizes': (100, 50), 'mlp3__hidden_layer_sizes': (80, 30)}

Os resultados foram os seguintes:





Relatório de Classificação do melhor modelo do comitê de redes neurais:
              
              precision    recall  f1-score   support

               0       0.84      0.80      0.82        51
               1       0.74      0.78      0.76        37

        accuracy                           0.80        88
       macro avg       0.79      0.79      0.79        88
    weighted avg       0.80      0.80      0.80        88


## MLP

### Base de dados normalizada

Nessas condições o melhor resultado foi obtido usando os parâmetros obtidos pelo 1o GridSearch, que deu o seguinte resultado:


- Melhores parâmetros encontrados: {'hidden_layer_sizes': (50, 20)}

Relatório de Classificação do melhor modelo:
              
                  precision    recall  f1-score   support

               0       0.98      0.98      0.98      1313
               1       0.45      0.35      0.40        51

        accuracy                           0.96      1364
       macro avg       0.71      0.67      0.69      1364
    weighted avg       0.96      0.96      0.96      1364

### Base de dados com oversampling

Nesse database o melhor resultado foi enctrado pelo segundo GridSearch feito. Os resultados foram os seguintes:


- Melhores parâmetros para o comitê de redes neurais: {'alpha': 0.0001, 'learning_rate_init': 0.001}

Relatório de Classificação do melhor modelo do comitê de redes neurais:
  
                    precision    recall  f1-score   support

               0       1.00      0.98      0.99      1334
               1       0.98      1.00      0.99      1306

        accuracy                           0.99      2640
       macro avg       0.99      0.99      0.99      2640
    weighted avg       0.99      0.99      0.99      2640

### Base de dados com undersampling

Nesse caso o melhor conjunto de parâmetros foi encontrado no 1o GridSearch. O resultado foi o seguinte:


- Melhores parâmetros encontrados: {'hidden_layer_sizes': (50, 20)}

Relatório de Classificação do melhor modelo:
              
                    precision    recall  f1-score   support

               0       0.87      0.78      0.82        51
               1       0.74      0.84      0.78        37

        accuracy                           0.81        88
       macro avg       0.80      0.81      0.80        88
    weighted avg       0.81      0.81      0.81        88

## KNN

### Base de dados normalizada

Nesse contexto o melhor resultado foi encontrado pelo 1o GridSearch, com a seguinte configuração (vale notar que o 2o GridSearch achou o mesmo resultado):

- Melhores parâmetros encontrados: {'n_neighbors': 7}

Relatório de Classificação do melhor modelo:
              
                    precision    recall  f1-score   support

               0       0.97      1.00      0.98      1313
               1       0.67      0.12      0.20        51

        accuracy                           0.96      1364
       macro avg       0.82      0.56      0.59      1364
    weighted avg       0.96      0.96      0.95      1364

### Base de dados com oversampling

Nesse caso o melhor resultado foi obtido com os parâmetros encontrados no 1o GridSearch com a seguinte configuração:

- Melhores parâmetros encontrados: {'n_neighbors': 2}

Relatório de Classificação do melhor modelo:
              
                    precision    recall  f1-score   support

               0       1.00      0.94      0.97      1334
               1       0.95      1.00      0.97      1306

        accuracy                           0.97      2640
       macro avg       0.97      0.97      0.97      2640
    weighted avg       0.97      0.97      0.97      2640


### Base de dados com undersampling

Nesse caso o melhor resultado foi obtido com o segundo GridSearch com a seguinte configuração:


- Melhores parâmetros encontrados: {'p': 1, 'weights': 'uniform'}

Relatório de Classificação do melhor modelo:

                    precision    recall  f1-score   support

               0       0.88      0.82      0.85        51
               1       0.78      0.84      0.81        37

        accuracy                           0.83        88
       macro avg       0.82      0.83      0.83        88
    weighted avg       0.83      0.83      0.83        88